In [10]:
# Install the required packages
%pip install opencv-python-headless numpy torch

import cv2
import numpy as np
import torch
import time
from collections import defaultdict


from sort.sort import Sort  # Import the SORT tracker

# Load the YOLOv5 model (YOLOv5m is used for better accuracy)
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# COCO class labels with custom names
class_labels = {
    2: 'Car',
    3: 'Motorbike',
    5: 'Bus',
    7: 'Truck',
    0: 'Person',
    1: 'Bicycle',
    9: 'Three-wheeler',  # Placeholder for custom class (if available)
}

# Dictionary to track object appearance times and first seen timestamps
active_tracks = {}

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Function to log data to a text file when the object disappears
def log_to_file(object_id, label, duration, first_seen, last_seen):
    with open("tracking_log.txt", "a") as file:
        file.write(f"ID: {object_id}, Type: {label}, Duration: {duration:.2f} seconds, First seen: {first_seen}, Last seen: {last_seen}\n")
# Initialize SORT tracker
tracker = Sort()
# Initialize SORT tracker
tracker = SORT()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally to correct axis
    frame = cv2.flip(frame, 1)

    # Object detection using YOLOv5
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes (x1, y1, x2, y2, confidence, class)

    # Filter vehicle and person detections
    vehicle_classes = [2, 3, 5, 7, 1, 9]  # Car, Motorbike, Bus, Truck, Bicycle, Three-wheeler
    vehicles = [d for d in detections if int(d[5]) in vehicle_classes and d[4] > 0.4]
    people = [d for d in detections if int(d[5]) == 0 and d[4] > 0.4]  # Detect people (class 0)

    # Combine vehicles and people detections
    detections = vehicles + people

    # Track current frame's object IDs
    current_frame_ids = set()

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        label = class_labels.get(int(cls), 'Unknown')
        object_id = int(x1) + int(y1)  # Simple way to generate ID (replace with better tracking logic)

        current_frame_ids.add(object_id)

        # If the object is new, record the appearance time
        if object_id not in active_tracks:
            active_tracks[object_id] = {'first_seen': time.time(), 'label': label}

        # Draw bounding box in green
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Display ID and label in red
        cv2.putText(frame, f'{label} ID: {object_id}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Check for objects that have left the frame
    for object_id in list(active_tracks.keys()):
        if object_id not in current_frame_ids:
            first_seen = active_tracks[object_id]['first_seen']
            last_seen = time.time()
            duration = last_seen - first_seen
            log_to_file(object_id, active_tracks[object_id]['label'], duration, first_seen, last_seen)  # Log data to file
            del active_tracks[object_id]  # Remove the object from active tracks

    # Display the frame with tracking
    cv2.imshow('Real-Time Multi-Object Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Krithik R\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\cv2\\cv2.pyd'
Check the permissions.


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Krithik R\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'skimage'